### Import dependencies

In [21]:
# Dependencies and Setup
import os
import pandas as pd
import numpy as np
import json
import requests

### Import Suicide rate and financial indicators for several years

In [22]:
# Specify CSV file containing master data
# Data Source: 
# https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016

master_file = os.path.join(".","Data/master.csv")

# Import CSV file as Dataframe
master_df = pd.read_csv(master_file)
master_df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


### Add ISO 3 Codes to country names and clean up (makes merging alot easier later on!)

In [23]:
# Use REST API to collect ISO3code listing for all countries
code_url = "https://restcountries.eu/rest/v2/all"

# Make JSON request to Rest Countries API
country_data = requests.get(code_url).json()

# Peek JSON formatted data
print(json.dumps(country_data,indent=4, sort_keys=True))


[
    {
        "alpha2Code": "AF",
        "alpha3Code": "AFG",
        "altSpellings": [
            "AF",
            "Af\u0121\u0101nist\u0101n"
        ],
        "area": 652230.0,
        "borders": [
            "IRN",
            "PAK",
            "TKM",
            "UZB",
            "TJK",
            "CHN"
        ],
        "callingCodes": [
            "93"
        ],
        "capital": "Kabul",
        "cioc": "AFG",
        "currencies": [
            {
                "code": "AFN",
                "name": "Afghan afghani",
                "symbol": "\u060b"
            }
        ],
        "demonym": "Afghan",
        "flag": "https://restcountries.eu/data/afg.svg",
        "gini": 27.8,
        "languages": [
            {
                "iso639_1": "ps",
                "iso639_2": "pus",
                "name": "Pashto",
                "nativeName": "\u067e\u069a\u062a\u0648"
            },
            {
                "iso639_1": "uz",
                "iso639_2

In [24]:
# Determine how many countries are included in this data set
nCountriesCodes = len(country_data)

# Initialize lists
nameCountry = []       # name of country
a2cCountry = []        # ISO 3166-1 2-letter country code
a3cCountry = []        # ISO 3166-1 3-letter country code
latlng = []            # geocache data

# Start gathering data for all countries. 
# If a country's data can not be obtained then skip entry and move on to the next...

for code in np.arange(0,nCountriesCodes,1):
    
    try:
        nameCountry.append(country_data[code]["name"])
        a3cCountry.append(country_data[code]["alpha3Code"])
        a2cCountry.append(country_data[code]["alpha2Code"])
        latlng.append(country_data[code]["latlng"])

    except:
        print(f"Country Not Listed in index {code}... skipping.")

In [25]:
# Construct dataframe with country, code, and geocoordinates
raw_codes_df = pd.DataFrame({"Country":nameCountry,
                                "Code":a3cCountry,
                                ##"2Code":a2cCountry,
                                "Location":latlng})
raw_codes_df.head(3)

,Country,Code,Location
0,Afghanistan,AFG,"[33.0, 65.0]"
1,Åland Islands,ALA,"[60.116667, 19.9]"
2,Albania,ALB,"[41.0, 20.0]"


In [26]:
# Add ISO country codes and location of country to master_df
geoMaster = master_df.merge(raw_codes_df,left_on= "country",right_on="Country",how="left")
geoMaster

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation,Country,Code,Location
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X,Albania,ALB,"[41.0, 20.0]"
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent,Albania,ALB,"[41.0, 20.0]"
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X,Albania,ALB,"[41.0, 20.0]"
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation,Albania,ALB,"[41.0, 20.0]"
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers,Albania,ALB,"[41.0, 20.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,107,3620833,2.96,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation X,Uzbekistan,UZB,"[41.0, 64.0]"
27816,Uzbekistan,2014,female,75+ years,9,348465,2.58,Uzbekistan2014,0.675,"63,067,077,179",2309,Silent,Uzbekistan,UZB,"[41.0, 64.0]"
27817,Uzbekistan,2014,male,5-14 years,60,2762158,2.17,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z,Uzbekistan,UZB,"[41.0, 64.0]"
27818,Uzbekistan,2014,female,5-14 years,44,2631600,1.67,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z,Uzbekistan,UZB,"[41.0, 64.0]"


In [27]:
# The following countries were not merged correctly (i.e., country names differ in spelling)
unfoundCountries = geoMaster.loc[pd.isnull(geoMaster["Code"])]
print(f"List of unmatched coutries:\n{unfoundCountries['country'].unique()}")

List of unmatched coutries:
['Macau' 'Republic of Korea' 'Saint Vincent and Grenadines'
 'United Kingdom' 'United States']


In [28]:
# Print list of troublesome countries (countries with no name matches)
print(raw_codes_df[raw_codes_df['Country'].str.contains("States")]["Country"])

33     United States Minor Outlying Islands
145        Micronesia (Federated States of)
239                United States of America
Name: Country, dtype: object


In [29]:
# Correct master data to facilitate finding ISO-codes
corrections = {"Macau":"Macao",
              "Republic of Korea":"Korea (Republic of)",
              "Saint Vincent and Grenadines":"Saint Vincent and the Grenadines",
              "United Kingdom":"United Kingdom of Great Britain and Northern Ireland",
              "United States":"United States of America"}

# Make name corrections master data set
for country in corrections:
    oldName = country
    newName = corrections[country]
    master_df.loc[master_df["country"] == oldName,"country"] = newName

In [30]:
# Make the merge again (after correction)
geoMaster = master_df.merge(raw_codes_df,left_on= "country",right_on="Country",how="left")
geoMaster

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation,Country,Code,Location
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X,Albania,ALB,"[41.0, 20.0]"
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent,Albania,ALB,"[41.0, 20.0]"
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X,Albania,ALB,"[41.0, 20.0]"
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation,Albania,ALB,"[41.0, 20.0]"
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers,Albania,ALB,"[41.0, 20.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,107,3620833,2.96,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation X,Uzbekistan,UZB,"[41.0, 64.0]"
27816,Uzbekistan,2014,female,75+ years,9,348465,2.58,Uzbekistan2014,0.675,"63,067,077,179",2309,Silent,Uzbekistan,UZB,"[41.0, 64.0]"
27817,Uzbekistan,2014,male,5-14 years,60,2762158,2.17,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z,Uzbekistan,UZB,"[41.0, 64.0]"
27818,Uzbekistan,2014,female,5-14 years,44,2631600,1.67,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z,Uzbekistan,UZB,"[41.0, 64.0]"


### Start of Data Extraction

In [31]:
# Ensure we have a complete match!
unfoundCountries = geoMaster.loc[pd.isnull(geoMaster["Code"])]
print(f"List of unmatched coutries:\n{unfoundCountries['country'].unique()}")

List of unmatched coutries:
[]


In [32]:
# List number of different countries
noCountries = len(geoMaster["country"].unique())
noCountries

101

In [33]:
# List of different countries
nameCountries = geoMaster["country"].unique()
nameCountries

array(['Albania', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Barbados', 'Belarus', 'Belgium', 'Belize',
       'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Cabo Verde',
       'Canada', 'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba',
       'Cyprus', 'Czech Republic', 'Denmark', 'Dominica', 'Ecuador',
       'El Salvador', 'Estonia', 'Fiji', 'Finland', 'France', 'Georgia',
       'Germany', 'Greece', 'Grenada', 'Guatemala', 'Guyana', 'Hungary',
       'Iceland', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan',
       'Kazakhstan', 'Kiribati', 'Kuwait', 'Kyrgyzstan', 'Latvia',
       'Lithuania', 'Luxembourg', 'Macao', 'Maldives', 'Malta',
       'Mauritius', 'Mexico', 'Mongolia', 'Montenegro', 'Netherlands',
       'New Zealand', 'Nicaragua', 'Norway', 'Oman', 'Panama', 'Paraguay',
       'Philippines', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar',
       'Korea (Republic of)', 'Romania',

In [34]:
# Determine suicide totals per year per country
total_suicides = geoMaster.groupby(["Country","Code","year"]).sum()[["suicides_no","population"]]
total_suicides.head()

suicides_no  population
Country Code year                         
Albania ALB  1987           73     2709600
             1988           63     2764300
             1989           68     2803100
             1992           47     2822500
             1993           73     2807300

In [35]:
# Determine suicide rate (per 100k) for the entire year
tot = [];
for i in np.arange(0,len(total_suicides["suicides_no"])):
    tot.append(round(total_suicides["suicides_no"][i]/total_suicides["population"][i]*100000,2))

In [36]:
# Add total suicide rate to dataframe
total_suicides["total suicides/100k pop"] = tot

In [37]:
# Display total suicides dataframe
total_suicides.head()

suicides_no  population  total suicides/100k pop
Country Code year                                                  
Albania ALB  1987           73     2709600                     2.69
             1988           63     2764300                     2.28
             1989           68     2803100                     2.43
             1992           47     2822500                     1.67
             1993           73     2807300                     2.60

In [38]:
# Let's collapse the mulitindices
flip = total_suicides.reset_index()
flip.head(30)
flip_output = os.path.join(".","Data/suicide_data.csv")
flip.to_csv(flip_output, index_label="index")

In [39]:
# Retrive the latest year when suicide data was recorded
countryLastDate = flip[["Country","Code","year"]].groupby(["Country","Code"]).max()
countryLastDate= countryLastDate.reset_index()
countryLastDate

,Country,Code,year
0,Albania,ALB,2010
1,Antigua and Barbuda,ATG,2015
2,Argentina,ARG,2015
3,Armenia,ARM,2016
4,Aruba,ABW,2011
...,...,...,...
96,United Arab Emirates,ARE,2010
97,United Kingdom of Great Britain and Northern I...,GBR,2015
98,United States of America,USA,2015
99,Uruguay,URY,2015


In [40]:
# Testing out how multiindex references work...
total_suicides.loc[('Albania', 'ALB', 1987)]

suicides_no                     73.00
population                 2709600.00
total suicides/100k pop          2.69
Name: (Albania, ALB, 1987), dtype: float64

In [41]:
# For the last year when data was recorded (per country) pull the corresponding
# number of suicides, population, and suicide rate
latestSuicideNo = []
latestPop = []
latestSuicideRate = []
for i in countryLastDate.index:
    country = countryLastDate.loc[i]['Country']
    code = countryLastDate.loc[i]['Code']
    latest_year = countryLastDate.loc[i]['year']
    entry = total_suicides.loc[(country, code, latest_year)]
    latestSuicideNo.append(entry['suicides_no'])
    latestPop.append(entry['population'])
    latestSuicideRate.append(entry['total suicides/100k pop'])

In [42]:
# Add latest data LastDate dataframe
countryLastDate["suicides_no"] = latestSuicideNo
countryLastDate["population"] = latestPop
countryLastDate["total suicides/100k pop"] = latestSuicideRate
countryLastDate

,Country,Code,year,suicides_no,population,total suicides/100k pop
0,Albania,ALB,2010,96.0,2736025.0,3.51
1,Antigua and Barbuda,ATG,2015,1.0,91889.0,1.09
2,Argentina,ARG,2015,3073.0,39699624.0,7.74
3,Armenia,ARM,2016,67.0,2400033.0,2.79
4,Aruba,ABW,2011,9.0,95488.0,9.43
...,...,...,...,...,...,...
96,United Arab Emirates,ARE,2010,111.0,7839736.0,1.42
97,United Kingdom of Great Britain and Northern I...,GBR,2015,4910.0,61082942.0,8.04
98,United States of America,USA,2015,44189.0,300078511.0,14.73
99,Uruguay,URY,2015,630.0,3190795.0,19.74


In [43]:
# no of countries with data available for years 2015 and 2016
#idx = pd.IndexSlice
#len(total_suicides.loc[idx[:,:, [2015,2016]], :].index.unique(level='Country'))

In [44]:
# Pull suicide rates for 2015 and 2016 only
#suicideRate2015 = total_suicides.loc[idx[:,:, [2015]], :]["total suicides/100k pop"].to_frame().reset_index()
#suicideRate2015 = suicideRate2015.drop(columns=["Country","year"])

In [45]:
##total_suicides.loc[(slice(None), slice(None), [2015,2016]), :]
## this code also works, just commented out but wanted to keep as notes

In [64]:
# Create suicide pivot table
suicidePivot = pd.pivot_table(geoMaster, 
                             values=["suicides/100k pop"], 
                             index=["Country","Code","year"],
                             columns=["age", "sex"])

In [47]:
# Display the pivot table
suicidePivot

suicides/100k pop                                        \
age                        15-24 years        25-34 years        35-54 years   
sex                             female   male      female   male      female   
Country    Code year                                                           
Albania    ALB  1987              4.83   6.71        1.56   3.28        2.15   
                1988              2.71   5.33        1.91   1.79        1.41   
                1989              1.67   3.71        0.75   6.35        2.43   
                1992              2.39   3.41        1.50   2.85        0.62   
                1993              3.51   7.40        2.67   3.91        2.11   
...                                ...    ...         ...    ...         ...   
Uzbekistan UZB  2010              6.71   7.67        3.89   9.89        3.02   
                2011              8.02   8.22        5.63  11.03        2.29   
                2012             10.84   9.72        5.79  11.38        2.56   
                2013             10.63  10.09        5.51  12.40        2.79   
                2014             11.59  11.10        5.92  11.61        2.96   

                                                                           \
age                         5-14 years       55-74 years        75+ years   
sex                    male     female  male      female   male    female   
Country    Code year                                                        
Albania    ALB  1987   5.19       0.00  0.00        0.00   0.73      2.81   
                1988   4.46       0.00  0.00        2.03   2.85      5.49   
                1989   4.71       0.00  0.00        0.67   4.22      0.00   
                1992   3.49       0.00  0.00        0.61   3.13      0.00   
                1993   2.85       0.29  0.27        1.18   4.24      0.00   
...                     ...        ...   ...         ...    ...       ...   
Uzbekistan UZB  2010  11.81       1.14  1.74        1.38   9.84      1.55   
                2011  14.26       1.08  1.66        1.56   7.00      1.51   
                2012  12.34       0.62  2.48        1.95  11.65      1.18   
                2013  14.37       1.19  2.24        1.55   9.89      2.32   
                2014  15.17       1.67  2.17        1.46  11.33      2.58   

                            
age                         
sex                   male  
Country    Code year        
Albania    ALB  1987  4.59  
                1988  4.48  
                1989  8.89  
                1992  0.00  
                1993  4.13  
...                    ...  
Uzbekistan UZB  2010  4.59  
                2011  4.88  
                2012  7.02  
                2013  5.88  
                2014  7.56  

[2321 rows x 12 columns]

In [48]:
# Add a column
suicidePivot[('suicides/100k pop',"All Ages","both")]=total_suicides["total suicides/100k pop"]

In [49]:
# Display the pivot table
suicidePivot.head()

suicides/100k pop                                            \
age                     15-24 years       25-34 years       35-54 years         
sex                          female  male      female  male      female  male   
Country Code year                                                               
Albania ALB  1987              4.83  6.71        1.56  3.28        2.15  5.19   
             1988              2.71  5.33        1.91  1.79        1.41  4.46   
             1989              1.67  3.71        0.75  6.35        2.43  4.71   
             1992              2.39  3.41        1.50  2.85        0.62  3.49   
             1993              3.51  7.40        2.67  3.91        2.11  2.85   

                                                                               
age               5-14 years       55-74 years       75+ years       All Ages  
sex                   female  male      female  male    female  male     both  
Country Code year                                                              
Albania ALB  1987       0.00  0.00        0.00  0.73      2.81  4.59     2.69  
             1988       0.00  0.00        2.03  2.85      5.49  4.48     2.28  
             1989       0.00  0.00        0.67  4.22      0.00  8.89     2.43  
             1992       0.00  0.00        0.61  3.13      0.00  0.00     1.67  
             1993       0.29  0.27        1.18  4.24      0.00  4.13     2.60

### Convert Suicide Table to a JSON

In [61]:
# Initialize JSON
sJSON = [];

# Create a list of different age groups (all is not an age group)
ageGrps = list(suicidePivot.columns.levels[1])

# Construct a dictionary with the following fields:
# country, code, year, male (rates), female (rates), both (rate)
for i in suicidePivot.index:
    iJSON = {}
    iJSON["country"] = i[0]
    iJSON["code"]= i[1]
    iJSON["year"]=i[2]
    mData = {}
    fData = {}
    bData = {}
    for g in ageGrps[0:len(ageGrps)-1]:
        if pd.notna(suicidePivot.loc[(i[0],i[1],i[2])][("suicides/100k pop",g,"male")]):
            mData[g] = suicidePivot.loc[(i[0],i[1],i[2])][("suicides/100k pop",g,"male")]
        if pd.notna(suicidePivot.loc[(i[0],i[1],i[2])][("suicides/100k pop",g,"female")]):
            fData[g] = suicidePivot.loc[(i[0],i[1],i[2])][("suicides/100k pop",g,"female")]
    iJSON["male"] = mData
    iJSON["female"] = fData
    bData["all ages"] = suicidePivot.loc[(i[0],i[1],i[2])][("suicides/100k pop","All Ages","both")]
    iJSON["both"] = bData
    sJSON.append(iJSON)

In [62]:
#Display JSON
sJSON

[{'country': 'Albania',
  'code': 'ALB',
  'year': 1987,
  'male': {'15-24 years': 6.71,
   '25-34 years': 3.28,
   '35-54 years': 5.19,
   '5-14 years': 0.0,
   '55-74 years': 0.73,
   '75+ years': 4.59},
  'female': {'15-24 years': 4.83,
   '25-34 years': 1.56,
   '35-54 years': 2.15,
   '5-14 years': 0.0,
   '55-74 years': 0.0,
   '75+ years': 2.81},
  'both': {'all ages': 2.69}},
 {'country': 'Albania',
  'code': 'ALB',
  'year': 1988,
  'male': {'15-24 years': 5.33,
   '25-34 years': 1.79,
   '35-54 years': 4.46,
   '5-14 years': 0.0,
   '55-74 years': 2.85,
   '75+ years': 4.48},
  'female': {'15-24 years': 2.71,
   '25-34 years': 1.91,
   '35-54 years': 1.41,
   '5-14 years': 0.0,
   '55-74 years': 2.03,
   '75+ years': 5.49},
  'both': {'all ages': 2.28}},
 {'country': 'Albania',
  'code': 'ALB',
  'year': 1989,
  'male': {'15-24 years': 3.71,
   '25-34 years': 6.35,
   '35-54 years': 4.71,
   '5-14 years': 0.0,
   '55-74 years': 4.22,
   '75+ years': 8.89},
  'female': {'15-24

In [63]:
# Specify CSV file containing happiness data
sJSON_output = os.path.join(".","Data/suicide_data.json")

with open(sJSON_output, 'w') as f:
    json.dump(sJSON, f)

### Gather financial data

In [33]:
# Pull data for financial indicators
financial_df = geoMaster[["Country","Code","year","HDI for year"," gdp_for_year ($) ","gdp_per_capita ($)"]]


In [34]:
# Change column name and remove extra weird spaces
financial_df = financial_df.rename(columns={" gdp_for_year ($) ":"gdp_for_year ($)"})

In [35]:
# Yearly data occurs many times, just provide a single occurence and drop duplicates
financial_df.drop_duplicates(inplace = True)
financial_df = financial_df.set_index(["Country","Code","year"])
financial_df

HDI for year gdp_for_year ($)  gdp_per_capita ($)
Country    Code year                                                   
Albania    ALB  1987           NaN    2,156,624,900                 796
                1988           NaN    2,126,000,000                 769
                1989           NaN    2,335,124,988                 833
                1992           NaN      709,452,584                 251
                1993           NaN    1,228,071,038                 437
...                            ...              ...                 ...
Uzbekistan UZB  2010         0.655   39,332,770,929                1533
                2011         0.661   45,915,191,189                1767
                2012         0.668   51,821,573,338                1964
                2013         0.672   57,690,453,461                2150
                2014         0.675   63,067,077,179                2309

[2321 rows x 3 columns]

In [36]:
# Pull the latest data for the provided financial indicators given the last year
# suicide years were recorded, per country
latestGDP = []
latestGDPcap = []
for i in countryLastDate.index:
    country = countryLastDate.loc[i]['Country']
    code = countryLastDate.loc[i]['Code']
    latest_year = countryLastDate.loc[i]['year']
    entry = financial_df.loc[(country, code, latest_year)]
    latestGDP.append(entry['gdp_for_year ($)'])
    latestGDPcap.append(entry['gdp_per_capita ($)'])

In [37]:
# Append financial data to current data
countryLastDate["gdp_for_year ($)"] = latestGDP
countryLastDate["gdp_per_capita ($)"] = latestGDPcap
countryLastDate

,Country,Code,year,suicides_no,population,total suicides/100k pop,gdp_for_year ($),gdp_per_capita ($)
0,Albania,ALB,2010,96.0,2736025.0,3.51,"11,926,953,259",4359
1,Antigua and Barbuda,ATG,2015,1.0,91889.0,1.09,"1,364,863,037",14853
2,Argentina,ARG,2015,3073.0,39699624.0,7.74,"594,749,285,413",14981
3,Armenia,ARM,2016,67.0,2400033.0,2.79,"10,546,135,160",3788
4,Aruba,ABW,2011,9.0,95488.0,9.43,"2,584,463,687",27066
...,...,...,...,...,...,...,...,...
96,United Arab Emirates,ARE,2010,111.0,7839736.0,1.42,"289,787,338,325",36964
97,United Kingdom of Great Britain and Northern I...,GBR,2015,4910.0,61082942.0,8.04,"2,885,570,309,161",47240
98,United States of America,USA,2015,44189.0,300078511.0,14.73,"18,120,714,000,000",60387
99,Uruguay,URY,2015,630.0,3190795.0,19.74,"53,274,304,222",16696


In [38]:
# Construct financial data for 2015 and 2016
#fin2015 = financial_df.loc[financial_df["year"] == 2015][["Code","gdp_for_year ($)","gdp_per_capita ($)"]]
#fin2015 = fin2015.rename(columns={"gdp_for_year ($)":"gdp_for_2015 ($)","gdp_per_capita ($)":"gdp_per_capita_2015 ($)"})
#
#fin2016 = financial_df.loc[financial_df["year"] == 2016][["Code","gdp_for_year ($)","gdp_per_capita ($)"]]
#fin2016 = fin2016.rename(columns={"gdp_for_year ($)":"gdp_for_2016 ($)","gdp_per_capita ($)":"gdp_per_capita_2016 ($)"})

### Gather happiness data

In [44]:
# Specify CSV file containing happiness data
happiness_2020 = os.path.join(".","Data/WHR20_DataForFigure2.1.csv")

# Import CSV file as Dataframe
happiness_2020_df = pd.read_csv(happiness_2020)

In [45]:
# Peek happiness data
happiness_2020_df

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.8087,0.031156,7.869766,7.747634,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
1,Denmark,Western Europe,7.6456,0.033492,7.711245,7.579955,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
2,Switzerland,Western Europe,7.5599,0.035014,7.628528,7.491272,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
3,Iceland,Western Europe,7.5045,0.059616,7.621347,7.387653,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
4,Norway,Western Europe,7.4880,0.034837,7.556281,7.419719,11.087804,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Central African Republic,Sub-Saharan Africa,3.4759,0.115183,3.701658,3.250141,6.625160,0.319460,45.200001,0.640881,0.082410,0.891807,1.972317,0.041072,0.000000,0.000000,0.292814,0.253513,0.028265,2.860198
149,Rwanda,Sub-Saharan Africa,3.3123,0.052425,3.415053,3.209547,7.600104,0.540835,61.098846,0.900589,0.055484,0.183541,1.972317,0.343243,0.522876,0.572383,0.604088,0.235705,0.485542,0.548445
150,Zimbabwe,Sub-Saharan Africa,3.2992,0.058674,3.414202,3.184198,7.865712,0.763093,55.617260,0.711458,-0.072064,0.810237,1.972317,0.425564,1.047835,0.375038,0.377405,0.151349,0.080929,0.841031
151,South Sudan,Sub-Saharan Africa,2.8166,0.107610,3.027516,2.605684,7.425360,0.553707,51.000000,0.451314,0.016519,0.763417,1.972317,0.289083,0.553279,0.208809,0.065609,0.209935,0.111157,1.378751


In [46]:
# Only pull happiness ranking and score
happiness_2020_df= happiness_2020_df[["Country name","Regional indicator","Ladder score"]].reset_index()
happiness_2020_df

,index,Country name,Regional indicator,Ladder score
0,0,Finland,Western Europe,7.8087
1,1,Denmark,Western Europe,7.6456
2,2,Switzerland,Western Europe,7.5599
3,3,Iceland,Western Europe,7.5045
4,4,Norway,Western Europe,7.4880
...,...,...,...,...
148,148,Central African Republic,Sub-Saharan Africa,3.4759
149,149,Rwanda,Sub-Saharan Africa,3.3123
150,150,Zimbabwe,Sub-Saharan Africa,3.2992
151,151,South Sudan,Sub-Saharan Africa,2.8166


In [47]:
# Rename columns
happiness_2020_df = happiness_2020_df.rename(columns={"index":"2020 Rank","Country name":"Country","Ladder score":"2020 Score","Regional indicator":"Region"})

In [48]:
# Make ranking start at 1st place (0th place, como que no...)
happiness_2020_df["2020 Rank"] +=1

In [49]:
# Rename columns prior to merging to current data
happiness_2020_df = happiness_2020_df[["Country","Region","2020 Rank","2020 Score"]]
happiness_2020_df

,Country,Region,2020 Rank,2020 Score
0,Finland,Western Europe,1,7.8087
1,Denmark,Western Europe,2,7.6456
2,Switzerland,Western Europe,3,7.5599
3,Iceland,Western Europe,4,7.5045
4,Norway,Western Europe,5,7.4880
...,...,...,...,...
148,Central African Republic,Sub-Saharan Africa,149,3.4759
149,Rwanda,Sub-Saharan Africa,150,3.3123
150,Zimbabwe,Sub-Saharan Africa,151,3.2992
151,South Sudan,Sub-Saharan Africa,152,2.8166


In [50]:
# Pull in country codes for countries listed in happiness report
happy2020master = happiness_2020_df.merge(raw_codes_df,on="Country",how="outer")
happy2020master 

,Country,Region,2020 Rank,2020 Score,Code,Location
0,Finland,Western Europe,1.0,7.8087,FIN,"[64.0, 26.0]"
1,Denmark,Western Europe,2.0,7.6456,DNK,"[56.0, 10.0]"
2,Switzerland,Western Europe,3.0,7.5599,CHE,"[47.0, 8.0]"
3,Iceland,Western Europe,4.0,7.5045,ISL,"[65.0, -18.0]"
4,Norway,Western Europe,5.0,7.4880,NOR,"[62.0, 10.0]"
...,...,...,...,...,...,...
265,Vanuatu,NaN,NaN,NaN,VUT,"[-16.0, 167.0]"
266,Venezuela (Bolivarian Republic of),NaN,NaN,NaN,VEN,"[8.0, -66.0]"
267,Viet Nam,NaN,NaN,NaN,VNM,"[16.16666666, 107.83333333]"
268,Wallis and Futuna,NaN,NaN,NaN,WLF,"[-13.3, -176.2]"


In [51]:
# The following countries were not merged correctly (i.e., country names differ in spelling)
unfoundCountries3 = happy2020master.loc[pd.isnull(happy2020master["Code"])]
print(f"List of unmatched coutries:\n{unfoundCountries3['Country'].unique()}")

List of unmatched coutries:
['United Kingdom' 'United States' 'Taiwan Province of China' 'Kosovo'
 'South Korea' 'Bolivia' 'Moldova' 'Russia' 'North Cyprus'
 'Hong Kong S.A.R. of China' 'Vietnam' 'Ivory Coast' 'Congo (Brazzaville)'
 'Macedonia' 'Venezuela' 'Laos' 'Iran' 'Palestinian Territories'
 'Congo (Kinshasa)' 'Tanzania']


In [52]:
# Print list of troublesome countries (countries with no name matches)
print(raw_codes_df[raw_codes_df['Country'].str.contains("Iv")]["Country"])

106    Côte d'Ivoire
Name: Country, dtype: object


In [53]:
# Correct master data to facilitate finding ISO-codes
# North Cyprus not officially recognized == has no ISO Code
corrections3 = {"Moldova":"Moldova (Republic of)",
                "Russia":"Russian Federation",
                "Kosovo":"Republic of Kosovo",
                "Vietnam":"Viet Nam",
                "Macedonia":"Macedonia (the former Yugoslav Republic of)",
                "Laos":"Lao People's Democratic Republic",
                "Palestinian Territories":"Palestine, State of",
                "Iran":"Iran (Islamic Republic of)",
                "Congo (Kinshasa)":"Congo (Democratic Republic of the)",
                "Congo (Brazzaville)":"Congo",
                "Ivory Coast": "Côte d'Ivoire",
                "Tanzania":"Tanzania, United Republic of",
                "Bolivia":"Bolivia (Plurinational State of)",
                "South Korea":"Korea (Democratic People's Republic of)",
                "Venezuela": "Venezuela (Bolivarian Republic of)",
                "Taiwan Province of China":"Taiwan",
                "Hong Kong S.A.R. of China":"Hong Kong",
                "United Kingdom":"United Kingdom of Great Britain and Northern Ireland",
                "United States":"United States of America"}

# Make name corrections master data set
for country in corrections3:
    oldName = country
    newName = corrections3[country]
    happiness_2020_df.loc[happiness_2020_df["Country"] == oldName,"Country"] = newName

In [54]:
# Merge again after the corrections
happy2020master = happiness_2020_df.merge(raw_codes_df,on="Country",how="left")
happy2020master 

,Country,Region,2020 Rank,2020 Score,Code,Location
0,Finland,Western Europe,1,7.8087,FIN,"[64.0, 26.0]"
1,Denmark,Western Europe,2,7.6456,DNK,"[56.0, 10.0]"
2,Switzerland,Western Europe,3,7.5599,CHE,"[47.0, 8.0]"
3,Iceland,Western Europe,4,7.5045,ISL,"[65.0, -18.0]"
4,Norway,Western Europe,5,7.4880,NOR,"[62.0, 10.0]"
...,...,...,...,...,...,...
148,Central African Republic,Sub-Saharan Africa,149,3.4759,CAF,"[7.0, 21.0]"
149,Rwanda,Sub-Saharan Africa,150,3.3123,RWA,"[-2.0, 30.0]"
150,Zimbabwe,Sub-Saharan Africa,151,3.2992,ZWE,"[-20.0, 30.0]"
151,South Sudan,Sub-Saharan Africa,152,2.8166,SSD,"[7.0, 30.0]"


In [55]:
# The following countries were not merged correctly (i.e., country names differ in spelling)
unfoundCountries3 = happy2020master.loc[pd.isnull(happy2020master["Code"])]
print(f"List of unmatched coutries:\n{unfoundCountries3['Country'].unique()}")

List of unmatched coutries:
['North Cyprus']


In [56]:
# Merge happiness data into dataframe with latest data
uberSet = countryLastDate.merge(happy2020master,on="Code",how="left")
uberSet=uberSet.drop(columns=["Country_y"])

### Complete data set!

In [59]:
# Drop missing happiness countries without suicide data
corrSet = uberSet.dropna(how="any")
corrSet = corrSet.rename(columns={"Country_x":"Country"})

In [61]:
# Specify CSV file containing happiness data
final_output = os.path.join(".","Data/latest_data_available.csv")

# Save CSV file with latest data for correlation
corrSet.to_csv(final_output)

### Experimental: Pull suicide data using WHO API

In [4]:
# Use REST API to collect ISO3code listing for all countries
who_url = "https://ghoapi.azureedge.net/api/MH_12"

# Make JSON request to Rest Countries API
who_data = requests.get(who_url).json()

# Peek JSON formatted data
print(json.dumps(who_data,indent=4, sort_keys=True))

{
    "@odata.context": "https://ghoapi.azureedge.net/api/$metadata#MH_12",
    "value": [
        {
            "Comments": null,
            "DataSourceDim": null,
            "DataSourceDimType": null,
            "Date": "2018-07-17T07:56:39.997+02:00",
            "Dim1": "FMLE",
            "Dim1Type": "SEX",
            "Dim2": null,
            "Dim2Type": null,
            "Dim3": null,
            "Dim3Type": null,
            "High": null,
            "Id": 19627179,
            "IndicatorCode": "MH_12",
            "Low": null,
            "NumericValue": 1.7,
            "SpatialDim": "AFG",
            "SpatialDimType": "COUNTRY",
            "TimeDim": 2000,
            "TimeDimType": "YEAR",
            "TimeDimensionBegin": "2000-01-01T00:00:00+01:00",
            "TimeDimensionEnd": "2000-12-31T00:00:00+01:00",
            "TimeDimensionValue": "2000",
            "Value": "1.7"
        },
        {
            "Comments": null,
            "DataSourceDim": null,
    